In [ ]:
%pip uninstall -y transformers accelerate
%pip install -U "transformers>=4.45" "accelerate>=0.30"

%pip install -U datasets
%pip install -U evaluate
%pip install -U sentencepiece protobuf tiktoken
%pip install -U scikit-learn
%pip install -U torch torchvision torchaudio




[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: pip install --upgr

In [2]:
import datasets
import inspect, os

print("datasets module file:", getattr(datasets, "__file__", None))
print("datasets version:", getattr(datasets, "__version__", None))
print("cwd:", os.getcwd())
print("dir contains datasets.py?", "datasets.py" in os.listdir("."))
print("dir contains datasets/ folder?", os.path.isdir("datasets"))


/Users/ElizabethMiriya/me-board/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


datasets module file: /Users/ElizabethMiriya/me-board/venv/lib/python3.12/site-packages/datasets/__init__.py
datasets version: 4.5.0
cwd: /Users/ElizabethMiriya/me-board/data-preprocessing
dir contains datasets.py? False
dir contains datasets/ folder? False


In [ ]:
import os
import re
import numpy as np
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)
import evaluate

# =====================
# CONFIG
# =====================
MODEL_NAME = "microsoft/deberta-v3-base"
CSV_PATH = "/content/drive/MyDrive/balanced_mental_health_data.csv"
OUTPUT_DIR = "./deberta_mh_subreddit"

TITLE_COL = "title"
BODY_COL = "selftext"
LABEL_COL = "subreddit"

MAX_LEN = 256
SEED = 42

# =====================
# LOAD CSV
# =====================
ds = load_dataset("csv", data_files={"data": CSV_PATH})["data"]

# =====================
# BASIC CLEANING
# =====================
REMOVED_PAT = re.compile(r"^\s*\[(removed|deleted)\]\s*$", re.IGNORECASE)

def clean_example(ex):
    title = (ex.get(TITLE_COL) or "").strip()
    body = (ex.get(BODY_COL) or "").strip()

    if REMOVED_PAT.match(title): title = ""
    if REMOVED_PAT.match(body): body = ""

    title = re.sub(r"\s+", " ", title).strip()
    body = re.sub(r"\s+", " ", body).strip()

    ex[TITLE_COL] = title
    ex[BODY_COL] = body
    return ex

ds = ds.map(clean_example)

def has_text(ex):
    return (len(ex[TITLE_COL]) + len(ex[BODY_COL])) > 0

ds = ds.filter(has_text)

# =====================
# LABEL MAPPING: subreddit -> int
# =====================
label_names = sorted(list(set(ds[LABEL_COL])))
label2id = {name: i for i, name in enumerate(label_names)}
id2label = {i: name for name, i in label2id.items()}

def map_label(ex):
    ex["labels"] = label2id[ex[LABEL_COL]]
    return ex

ds = ds.map(map_label)

num_labels = len(label_names)
print(f"Labels ({num_labels}): {label_names}")

# =====================
# TRAIN/VAL/TEST SPLIT
# =====================
ds = ds.shuffle(seed=SEED)
split1 = ds.train_test_split(test_size=0.2, seed=SEED)
split2 = split1["test"].train_test_split(test_size=0.5, seed=SEED)
datasets = DatasetDict({
    "train": split1["train"],
    "validation": split2["train"],
    "test": split2["test"],
})

# =====================
# TOKENIZATION
# =====================
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def build_text(title, body):
    title = (title or "").strip()
    body = (body or "").strip()
    if title and body:
        return f"[TITLE] {title}\n[BODY] {body}"
    if title:
        return f"[TITLE] {title}"
    return f"[BODY] {body}"

def tokenize_batched(batch):
    texts = [
        build_text(t, b)
        for t, b in zip(batch[TITLE_COL], batch[BODY_COL])
    ]
    return tokenizer(texts, truncation=True, max_length=MAX_LEN)

tokenized = datasets.map(tokenize_batched, batched=True)

# Remove unused columns
cols_to_remove = [c for c in [TITLE_COL, BODY_COL, LABEL_COL] if c in tokenized["train"].column_names]
tokenized = tokenized.remove_columns(cols_to_remove)

# Dynamic padding (this is where the tokenizer belongs now)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# =====================
# METRICS
# =====================
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"],
    }

# =====================
# MODEL
# =====================
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
)

model = model.float()

# =====================
# TRAINING ARGS
# =====================
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    seed=SEED,

    do_train=True,
    do_eval=True,
    eval_strategy="epoch",     # (some versions use evaluation_strategy)
    save_strategy="epoch",

    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,

    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    warmup_ratio=0.1,

    logging_steps=1,
    fp16=False,
    report_to="none",
)

# =====================
# TRAINER (NO tokenizer=...)
# =====================
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

batch = data_collator([tokenized["train"][i] for i in range(4)])
print(batch["labels"])
print(batch["input_ids"].shape)

outputs = model(**{k: v.to(model.device) for k, v in batch.items()})
print(f"Loss: {outputs.loss}")
print(f"Logits: {outputs.logits}")

trainer.train()

print("\nTEST SET EVAL:")
trainer.evaluate(tokenized["test"])

# Save both model + tokenizer
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"\nSaved to: {OUTPUT_DIR}")

Labels (5): ['Anxiety', 'SuicideWatch', 'depression', 'lonely', 'mentalhealth']


Loading weights: 100%|██████████| 198/198 [00:00<00:00, 2345.69it/s, Materializing param=deberta.encoder.rel_embeddings.weight]                     
DebertaV2ForSequenceClassification LOAD REPORT from: microsoft/deberta-v3-base
Key                                     | Status     | 
----------------------------------------+------------+-
lm_predictions.lm_head.dense.weight     | UNEXPECTED | 
mask_predictions.dense.weight           | UNEXPECTED | 
mask_predictions.dense.bias             | UNEXPECTED | 
mask_predictions.classifier.weight      | UNEXPECTED | 
mask_predictions.LayerNorm.bias         | UNEXPECTED | 
lm_predictions.lm_head.LayerNorm.weight | UNEXPECTED | 
mask_predictions.classifier.bias        | UNEXPECTED | 
lm_predictions.lm_head.dense.bias       | UNEXPECTED | 
lm_predictions.lm_head.bias             | UNEXPECTED | 
mask_predictions.LayerNorm.weight       | UNEXPECTED | 
lm_predictions.lm_head.LayerNorm.bias   | UNEXPECTED | 
classifier.weight                       | MI

Epoch,Training Loss,Validation Loss


RuntimeError: MPS backend out of memory (MPS allocated: 5.07 GiB, other allocations: 37.33 GiB, max allowed: 42.43 GiB). Tried to allocate 187.65 MiB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
!pip install -U huggingface_hub transformers safetensors
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

save_dir = "./deberta_mh_subreddit"

model.save_pretrained(save_dir, safe_serialization=True)
tokenizer.save_pretrained(save_dir)


In [ ]:
from huggingface_hub import create_repo
from transformers import AutoModelForSequenceClassification, AutoTokenizer

repo_id = "emngarcia/deberta_mh_subreddit"

create_repo(repo_id, exist_ok=True)

model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

In [ ]:
from transformers import pipeline

pipe = pipeline(
    "text-classification",
    model="emngarcia/deberta_mh_subreddit"
)

pipe("i hate living")